In [ ]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import os 
import numpy as np
import matplotlib.pyplot as plt
import cv2
import dlib

In [ ]:
def make_dataset():
    base_dir = r"E:\semester 5\mini project\face recognition algorithms\dataset"
    training_images_dir = [base_dir+'\\'+name for name in os.listdir(base_dir)]
    names = {}
    i = 0
    for p in os.listdir(base_dir):
        names[p] = i
        i+=1

    X = []
    y = []
    for person_dir in training_images_dir:
        for img in os.listdir(person_dir):
            image = cv2.imread(person_dir+'\\'+img)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (200,200))
            image = image/255
            X.append(image)
            y.append(names[person_dir.split('\\')[-1]])
    X = np.array(X)
    y = np.array(y)
    return X,y,names

In [ ]:
X,y,names = make_dataset()

In [ ]:
SHAPE = (200,200)

In [ ]:
pca = PCA()
pca.fit(np.reshape(X, (X.shape[0], 40000)))
plt.figure(figsize=(18, 7))
plt.plot(pca.explained_variance_ratio_.cumsum(), lw=3)

In [ ]:
np.where(pca.explained_variance_ratio_.cumsum() > 0.95)

In [ ]:
plt.imshow(pca.components_[4].reshape((200,200)))

In [ ]:
pca = PCA(n_components=62)
pca.fit(np.reshape(X, (X.shape[0], 40000)))

In [ ]:
x_train = pca.transform(np.reshape(X, (X.shape[0], 40000)))

In [ ]:
x_train.shape

In [ ]:
classifier = LogisticRegression(max_iter=500)
classifier.fit(x_train, y)

In [ ]:
classifier.score(x_train,y)

In [ ]:
def get_faces_from_image_2(img):
    detector = dlib.get_frontal_face_detector()
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_img)
    cropped_faces = []
    take = 7  # increase the size of the bounding box
    for face in faces:
        cropped_faces.append(img[ face.top()-take :face.bottom()+take, face.left()-take:face.right()+take , :].copy())
    return cropped_faces



In [ ]:
test_img = r"E:\semester 5\mini project\face recognition algorithms\Facenet\test.jpg"

In [ ]:
image = cv2.imread(test_img)
faces = get_faces_from_image_2(image)
len(faces)

In [ ]:
index = 11
print(faces[index].shape)
face = cv2.cvtColor(faces[index], cv2.COLOR_BGR2GRAY)
face = cv2.resize(face, (200,200))
face = face.reshape((40000))
face = np.array([face])
face = face/255
face = pca.transform(face)
plt.imshow(faces[index][:,:,::-1])
print(face.dtype)
prediction = classifier.predict(face)
print([k for k,v in names.items() if v==prediction])